In [296]:
import pandas as pd
import numpy as np

In [297]:
TM = pd.read_csv('TM_MDM_VHCLE_CMG (세종 차량연계 정보).csv')
VW_in = pd.read_csv('VW_InCarinfo (차량 입차 정보 조회).csv')
VW_in = VW_in.dropna()
VW_out = pd.read_csv('VW_OutCarinfo (차량 출차 정보 조회).csv')
car_normal = pd.read_csv('일반차량.csv')
car_ticket =pd.read_csv('정기권차량.csv')


import re
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['장비명2'][i] = reg
    return reg
VW_in['장비명2'] = 0
for i in range(len(VW_in)):
    slicing(VW_in,i)
VW_out['장비명2'] = 0
for i in range(len(VW_out)):
    slicing(VW_out,i)

arr = ['12동','13-1동','13-2동','13-3동','14-1동','14-2동','15동']
for i in arr :
    # num = len(VW_in[VW_in['장비명2']== i]) - len(VW_out[VW_out['장비명2']== i])
    # if num == 0 :
    #     print(f'{i}의 입차대수와 출차대수가 같습니다')
    # elif num > 0:
    #     print(f'{i}의 입차대수가 출차대수보다 {num}만큼 많습니다.')
    # else :
    #     print(f'{i}의 입차대수가 출차대수보다 {num}만큼 적습니다.')
    print(f'{i}의 입차 수' , len(VW_in[VW_in['장비명2']== i]), '출차 수',len(VW_out[VW_out['장비명2']== i]) )

VW_in['입차시간'] = VW_in['입차일자'] +  VW_in['입차시각']
VW_out['출차시간'] = VW_out['출차일자']  + VW_out['출차시각']
VW_in['입차시간'] = pd.to_datetime(VW_in['입차시간'], format='%Y-%m-%d %H:%M:%S', errors='raise')
VW_out['출차시간'] = pd.to_datetime(VW_out['출차시간'], format='%Y-%m-%d %H:%M:%S', errors='raise')

VW_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = VW_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
VW_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= VW_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)


/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_1781/45216043.py:2: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  VW_in = pd.read_csv('VW_InCarinfo (차량 입차 정보 조회).csv')
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_1781/45216043.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['장비명2'][i] = reg
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_1781/45216043.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['장비명2'][i] = reg


12동의 입차 수 9748 출차 수 8798
13-1동의 입차 수 4625 출차 수 4441
13-2동의 입차 수 4484 출차 수 3819
13-3동의 입차 수 839 출차 수 1053
14-1동의 입차 수 879 출차 수 1142
14-2동의 입차 수 11940 출차 수 13031
15동의 입차 수 6157 출차 수 6388


In [298]:
def pretreatment(df):
    df['일시'] = pd.to_datetime(df['FilePath'].apply(lambda x: x[12:26]), format='%Y%m%d%H%M%S')
    #df['일시'] = pd.to_datetime(df['일시']).apply(del_min_sec)
    df = df[['CarNo','일시', 'FilePath', 'InOutDiv']]
    df = df.rename({'FilePath':'주차일련번호'}, axis=1)
    return df
car_ticket = pretreatment(car_ticket)

In [299]:
car_ticket = car_ticket.sort_values(['CarNo','일시']).reset_index().drop(['index'],axis =1)
car_ticket['잔류시간'] = np.nan
car_ticket

,CarNo,일시,주차일련번호,InOutDiv,잔류시간
0,01노5776,2020-10-04 12:31:30,\2020\10\04\2020100412313001노5776F.jpg,1,NaN
1,01노5776,2020-10-04 19:05:57,\2020\10\04\2020100419055701노5776F.jpg,2,NaN
2,01노5776,2020-10-05 08:02:08,\2020\10\05\2020100508020801노5776F.jpg,1,NaN
3,01노5776,2020-10-05 13:19:50,\2020\10\05\2020100513195001노5776F.jpg,2,NaN
4,01노5776,2020-10-05 18:29:36,\2020\10\05\2020100518293601노5776F.jpg,1,NaN
...,...,...,...,...,...
42608,86고2448,2022-09-28 16:37:20,\2022\09\28\2022092816372086고2448F.jpg,1,NaN
42609,86고2448,2022-09-29 07:54:46,\2022\09\29\2022092907544686고2448F.jpg,2,NaN
42610,86고2448,2022-09-29 17:22:24,\2022\09\29\2022092917222486고2448F.jpg,1,NaN
42611,86고2448,2022-09-30 07:36:00,\2022\09\30\2022093007360086고2448F.jpg,2,NaN


In [300]:
def calculate_time(df):
    idx = df.index
    if idx < len(car_ticket):
        next_df = df.loc[idx+1]
        if (df['InOutDiv'] == 1) and  (next_df['InOutDiv'] == 2) and (df['CarNo'] == next_df['CarNo']):
            df['잔류시간'] = next_df['일시'] - df['일시']
            return df['잔류시간']



In [301]:
for idx in range(len(car_ticket)):
    if idx +1 ==len(car_ticket):
        break
    df = car_ticket.iloc[idx]
    next_df = car_ticket.iloc[idx+1]
    if (df['InOutDiv'] == 1) and  (next_df['InOutDiv'] == 2) and (df['CarNo'] == next_df['CarNo']):
        car_ticket.loc[idx,'잔류시간'] = next_df['일시'] - df['일시']
        car_ticket.loc[idx,'잔류시간(초)'] =  car_ticket.loc[idx,'잔류시간'].total_seconds()

In [302]:
car_ticket

,CarNo,일시,주차일련번호,InOutDiv,잔류시간,잔류시간(초)
0,01노5776,2020-10-04 12:31:30,\2020\10\04\2020100412313001노5776F.jpg,1,0 days 06:34:27,23667.0
1,01노5776,2020-10-04 19:05:57,\2020\10\04\2020100419055701노5776F.jpg,2,NaN,NaN
2,01노5776,2020-10-05 08:02:08,\2020\10\05\2020100508020801노5776F.jpg,1,0 days 05:17:42,19062.0
3,01노5776,2020-10-05 13:19:50,\2020\10\05\2020100513195001노5776F.jpg,2,NaN,NaN
4,01노5776,2020-10-05 18:29:36,\2020\10\05\2020100518293601노5776F.jpg,1,0 days 04:00:51,14451.0
...,...,...,...,...,...,...
42608,86고2448,2022-09-28 16:37:20,\2022\09\28\2022092816372086고2448F.jpg,1,0 days 15:17:26,55046.0
42609,86고2448,2022-09-29 07:54:46,\2022\09\29\2022092907544686고2448F.jpg,2,NaN,NaN
42610,86고2448,2022-09-29 17:22:24,\2022\09\29\2022092917222486고2448F.jpg,1,0 days 14:13:36,51216.0
42611,86고2448,2022-09-30 07:36:00,\2022\09\30\2022093007360086고2448F.jpg,2,NaN,NaN


In [303]:
import datetime
def replace_time(x):
    return x.replace('0 days ','')

time_delta = car_ticket.dropna()
#time_delta['잔류시간']= time_delta['잔류시간'].astype(str)
#time_delta['잔류시간']= time_delta['잔류시간'].apply(replace_time)

In [304]:
time_delta['잔류시간(분)'] = time_delta['잔류시간(초)']//60
time_delta['잔류시간(시)'] = time_delta['잔류시간(분)']//60 +1
time_delta

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_1781/2056455362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_delta['잔류시간(분)'] = time_delta['잔류시간(초)']//60
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_1781/2056455362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_delta['잔류시간(시)'] = time_delta['잔류시간(분)']//60 +1


,CarNo,일시,주차일련번호,InOutDiv,잔류시간,잔류시간(초),잔류시간(분),잔류시간(시)
0,01노5776,2020-10-04 12:31:30,\2020\10\04\2020100412313001노5776F.jpg,1,0 days 06:34:27,23667.0,394.0,7.0
2,01노5776,2020-10-05 08:02:08,\2020\10\05\2020100508020801노5776F.jpg,1,0 days 05:17:42,19062.0,317.0,6.0
4,01노5776,2020-10-05 18:29:36,\2020\10\05\2020100518293601노5776F.jpg,1,0 days 04:00:51,14451.0,240.0,5.0
6,01노5776,2020-10-06 07:51:39,\2020\10\06\2020100607513901노5776F.jpg,1,0 days 05:41:48,20508.0,341.0,6.0
8,01노5776,2020-10-07 08:03:36,\2020\10\07\2020100708033601노5776F.jpg,1,0 days 06:35:18,23718.0,395.0,7.0
...,...,...,...,...,...,...,...,...
42602,86고2448,2022-09-23 16:27:23,\2022\09\23\2022092316272386고2448F.jpg,1,2 days 15:07:36,227256.0,3787.0,64.0
42604,86고2448,2022-09-26 16:40:46,\2022\09\26\2022092616404686고2448F.jpg,1,0 days 14:42:57,52977.0,882.0,15.0
42606,86고2448,2022-09-27 17:37:26,\2022\09\27\2022092717372686고2448F.jpg,1,0 days 14:20:30,51630.0,860.0,15.0
42608,86고2448,2022-09-28 16:37:20,\2022\09\28\2022092816372086고2448F.jpg,1,0 days 15:17:26,55046.0,917.0,16.0


In [305]:
time_delta['잔류시간(시)'].describe()

count    21213.000000
mean         9.029039
std         29.447044
min          1.000000
25%          3.000000
50%          7.000000
75%         10.000000
max       2848.000000
Name: 잔류시간(시), dtype: float64

In [306]:
time_list = list(time_delta['잔류시간(시)'].unique())
time_list.sort()


In [307]:
arr_time = []
arr_num = []
for i in time_list:
    n = len(time_delta[time_delta['잔류시간(시)']==i])
    arr_time.append(i)
    arr_num.append(n)
x = pd.DataFrame({'시간' :arr_time,
                  '차량수':arr_num})
#x.to_excel('주차1동 주차2동 주치사간별 차량수.xlsx')

In [312]:
time_delta[time_delta['잔류시간(시)']>168]

,CarNo,일시,주차일련번호,InOutDiv,잔류시간,잔류시간(초),잔류시간(분),잔류시간(시)
3839,08주8743,2021-04-20 13:17:34,\2021\04\20\2021042013173408주8743F.jpg,1,10 days 00:10:27,864627.0,14410.0,241.0
3947,08주8743,2021-07-23 15:21:40,\2021\07\23\2021072315214008주8743F.jpg,1,8 days 15:55:43,748543.0,12475.0,208.0
3949,08주8743,2021-08-01 08:59:35,\2021\08\01\2021080108593508주8743F.jpg,1,7 days 01:01:04,608464.0,10141.0,170.0
3961,08주8743,2021-08-20 20:25:46,\2021\08\20\2021082020254608주8743F.jpg,1,118 days 15:39:43,10251583.0,170859.0,2848.0
3963,08주8743,2021-12-17 20:54:03,\2021\12\17\2021121720540308주8743F.jpg,1,15 days 21:49:13,1374553.0,22909.0,382.0
3967,08주8743,2022-01-08 23:54:28,\2022\01\08\2022010823542808주8743F.jpg,1,7 days 22:57:29,687449.0,11457.0,191.0
3969,08주8743,2022-01-16 22:56:43,\2022\01\16\2022011622564308주8743F.jpg,1,9 days 23:47:30,863250.0,14387.0,240.0
3975,08주8743,2022-02-01 14:23:46,\2022\02\01\2022020114234608주8743F.jpg,1,10 days 17:56:41,928601.0,15476.0,258.0
4061,08주8743,2022-05-30 00:38:46,\2022\05\30\2022053000384608주8743F.jpg,1,45 days 17:35:29,3951329.0,65855.0,1098.0
7848,104허2500,2022-05-29 23:09:51,\2022\05\29\20220529230951104허2500F.jpg,1,46 days 03:33:31,3987211.0,66453.0,1108.0


In [309]:
time_delta

,CarNo,일시,주차일련번호,InOutDiv,잔류시간,잔류시간(초),잔류시간(분),잔류시간(시)
0,01노5776,2020-10-04 12:31:30,\2020\10\04\2020100412313001노5776F.jpg,1,0 days 06:34:27,23667.0,394.0,7.0
2,01노5776,2020-10-05 08:02:08,\2020\10\05\2020100508020801노5776F.jpg,1,0 days 05:17:42,19062.0,317.0,6.0
4,01노5776,2020-10-05 18:29:36,\2020\10\05\2020100518293601노5776F.jpg,1,0 days 04:00:51,14451.0,240.0,5.0
6,01노5776,2020-10-06 07:51:39,\2020\10\06\2020100607513901노5776F.jpg,1,0 days 05:41:48,20508.0,341.0,6.0
8,01노5776,2020-10-07 08:03:36,\2020\10\07\2020100708033601노5776F.jpg,1,0 days 06:35:18,23718.0,395.0,7.0
...,...,...,...,...,...,...,...,...
42602,86고2448,2022-09-23 16:27:23,\2022\09\23\2022092316272386고2448F.jpg,1,2 days 15:07:36,227256.0,3787.0,64.0
42604,86고2448,2022-09-26 16:40:46,\2022\09\26\2022092616404686고2448F.jpg,1,0 days 14:42:57,52977.0,882.0,15.0
42606,86고2448,2022-09-27 17:37:26,\2022\09\27\2022092717372686고2448F.jpg,1,0 days 14:20:30,51630.0,860.0,15.0
42608,86고2448,2022-09-28 16:37:20,\2022\09\28\2022092816372086고2448F.jpg,1,0 days 15:17:26,55046.0,917.0,16.0
